In [42]:
import pandas as pd
import os
from pathlib import Path
import platform

#Directorio
directory = "HSConversionTables"
if platform.system()=='Windows':
    Path('C:/Users/Asus/Desktop/CEP/pedidos Igal/'+directory).mkdir(parents=True, exist_ok=True)
    os.chdir('C:/Users/Asus/Desktop/CEP/pedidos Igal'+directory)
else: 
    Path('/home/nachengue/Escritorio/CEP/'+directory).mkdir(parents=True, exist_ok=True)
    os.chdir('/home/nachengue/Escritorio/CEP/'+directory)
print(os.getcwd())

#Data
data = pd.read_excel("./data/CompleteCorrelationsOfHS-SITC-BEC_20170606.xlsx")
cols = ['HS92','HS96','HS02','HS07','HS12','HS17']
data = data[cols].dropna().drop_duplicates()
for col in cols: 
    data[col] = data[col].apply(lambda x: col+"-"+str(int(x)).zfill(6))
data['count'] = 1

/home/nachengue/Escritorio/CEP/HSConversionTables


In [43]:
def return_df(df,cat_cols=[],value_cols=''):
    
    labelList = []
    
    for catCol in cat_cols:
        labelListTemp =  df[catCol].unique().tolist()
        colorNumList.append(len(labelListTemp))
        labelList = labelList + labelListTemp
        
    # remove duplicates from labelList
    labelList = list(dict.fromkeys(labelList))
    
         
    # transform df into a source-target pair
    for i in range(len(cat_cols)-1):
        if i==0:
            sourceTargetDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            sourceTargetDf.columns = ['source','target','count']
        else:
            tempDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            tempDf.columns = ['source','target','count']
            sourceTargetDf = pd.concat([sourceTargetDf,tempDf])
    
    sourceTargetDf = sourceTargetDf.groupby(['source','target']).agg({'count':'sum'}).reset_index()
        
    # add index for source-target pair
    sourceTargetDf['sourceID'] = sourceTargetDf['source'].apply(lambda x: labelList.index(x)+1)
    sourceTargetDf['targetID'] = sourceTargetDf['target'].apply(lambda x: labelList.index(x)+1)

    return sourceTargetDf


def return_df2(df,cat_cols=[]):
    
    labelList = []
    
    for catCol in cat_cols:
        labelListTemp =  df[catCol].unique().tolist()
        labelList = labelList + labelListTemp
        
    # remove duplicates from labelList
    labelList = list(dict.fromkeys(labelList))
    
         
    # transform df into a source-target pair
    for i in range(len(cat_cols)-1):
        if i==0:
            sourceTargetDf = df[[cat_cols[i],cat_cols[i+1]]]
            sourceTargetDf.columns = ['source','target']
        else:
            tempDf = df[[cat_cols[i],cat_cols[i+1]]]
            tempDf.columns = ['source','target']
            sourceTargetDf = pd.concat([sourceTargetDf,tempDf])
    
    sourceTargetDf = sourceTargetDf.drop_duplicates()
        
    # add index for source-target pair starting from 1
    sourceTargetDf['sourceID'] = sourceTargetDf['source'].apply(lambda x: labelList.index(x)+1)
    sourceTargetDf['targetID'] = sourceTargetDf['target'].apply(lambda x: labelList.index(x)+1)

    return sourceTargetDf

In [44]:
df = return_df2(data, cols)
df.target.value_counts()

HS17-854231    159
HS17-854232    159
HS17-854239    159
HS17-854233     65
HS17-030299     41
              ... 
HS02-761100      1
HS02-761090      1
HS02-761010      1
HS02-760900      1
HS17-999999      1
Name: target, Length: 25986, dtype: int64

In [47]:
df[(df.target=="HS17-854231")&(df.source.str.contains("HS12-6"))]

,source,target,sourceID,targetID


In [5]:
#Function to return the used versions
import functools
from datetime import datetime

def return_used_versions(start_year = None, end_year=None):
    if start_year==None: 
        start_year = 1992
    if end_year==None: 
        end_year = datetime.now().year
    if start_year < 1992 or end_year<start_year or end_year>datetime.now().year: 
        print("Uncorrect years")
    else: 
        years = [1992, 1996, 2002, 2007, 2012, 2017]
        versions = ['HS92', 'HS96', 'HS02', 'HS07', 'HS12', 'HS17']
        version_dict = {k:v for k,v in zip(years, versions)}
        start_v = functools.reduce(lambda a,b: a if a<=start_year else b, list(reversed(years)))
        end_v = functools.reduce(lambda a,b: a if a<=end_year else b, list(reversed(years)))
        years_list = years[years.index(start_v):years.index(end_v)+1]
        return [version_dict[x] for x in years_list]



In [47]:
return_used_versions(2000,2015)

['HS96', 'HS02', 'HS07', 'HS12']

In [40]:
def get_outer_targets(data, position=None, start_year = None, end_year=None): 
    versions = return_used_versions(start_year, end_year)
    lversion = versions[-1]
    rel_ids = df[df.target==lversion+"-"+str(position)]['sourceID'].to_list()
    return df[df.sourceID.isin(rel_ids)]['target'].unique().tolist()

In [48]:
len(get_outer_targets(df, position=854231, start_year = 2015, end_year= 2020))

163

In [40]:
import pprint
from collections import defaultdict


class Graph(object):
    """ Graph data structure, undirected by default. """

    def __init__(self, connections, directed=False):
        self._graph = defaultdict(set)
        self._directed = directed
        self.add_connections(connections)

    def add_connections(self, connections):
        """ Add connections (list of tuple pairs) to graph """

        for node1, node2 in connections:
            self.add(node1, node2)

    def add(self, node1, node2):
        """ Add connection between node1 and node2 """

        self._graph[node1].add(node2)
        if not self._directed:
            self._graph[node2].add(node1)

    def remove(self, node):
        """ Remove all references to node """

        for n, cxns in self._graph.items():  # python3: items(); python2: iteritems()
            try:
                cxns.remove(node)
            except KeyError:
                pass
        try:
            del self._graph[node]
        except KeyError:
            pass

    def is_connected(self, node1, node2):
        """ Is node1 directly connected to node2 """

        return node1 in self._graph and node2 in self._graph[node1]

    def find_path(self, node1, node2, path=[]):
        """ Find any path between node1 and node2 (may not be shortest) """

        path = path + [node1]
        if node1 == node2:
            return path
        if node1 not in self._graph:
            return None
        for node in self._graph[node1]:
            if node not in path:
                new_path = self.find_path(node, node2, path)
                if new_path:
                    return new_path
        return None

    def find_all_connected_nodes(self, qnode, path=[]):
        
        path = path + [qnode]
        if qnode in path[:-1]:
            return path
        if qnode not in self._graph:
            return None
        for node in self._graph[qnode]:
            print(node)
            if node not in path:
                new_path = self.find_all_connected_nodes(node, path)
                if new_path:
                    return new_path
        return None
    
    def __str__(self):
        return '{}({})'.format(self.__class__.__name__, dict(self._graph))

In [5]:
data = pd.read_excel("./data/CompleteCorrelationsOfHS-SITC-BEC_20170606.xlsx")
cols = ['HS92','HS96','HS02','HS07','HS12','HS17']
data = data[cols].dropna().drop_duplicates()
for col in cols: 
    data[col] = data[col].apply(lambda x: col+"-"+str(int(x)).zfill(6))
connections = []
for i in range(len(cols)-1): 
        temp_tup_list = list(data[[cols[i], cols[i+1]]].drop_duplicates().itertuples(index=False, name=None))
        connections = connections + temp_tup_list

In [30]:
g = Graph(connections, directed=False)
pretty_print = pprint.PrettyPrinter()
# pretty_print.pprint(g._graph)

In [36]:
pretty_print.pprint(g.find_path("HS92-010111",'HS96-010120'))

['HS92-010111',
 'HS96-010111',
 'HS02-010110',
 'HS07-010110',
 'HS12-010130',
 'HS07-010190',
 'HS02-010190',
 'HS96-010120']


In [41]:
g.find_all_connected_nodes('HS96-010120')